In [ ]:
from datetime import datetime, timedelta
import pandas as pd
# import sqlalchemy

In [ ]:
# user_name = 'KHU9683'
# pword = 'Mer82@dme'
# host_name='EDWPROD.DW.MEDCITY.NET'

In [ ]:
# td_engine = sqlalchemy.create_engine('teradatasql://' + host_name + '/?user=' + user_name + '&password=' + pword + '&logmech=LDAP')

In [ ]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = '\n'.join(file_data)
        file.write(file_string)

In [ ]:
#path of file with table name and database name in csv file 
input_path = "InputFiles\\copy_tables.csv"
output_path = "OutputFiles\\copy_table_parameters.json"
testing_date = "2023-04-17"
test_environment = "qa"

In [ ]:
#get the DDLs for the tables from teradata 
def gen_copy_tables_parameters():
        dt1 = datetime.now()
        delimiter = ','
        quote_char = '"'
        df = pd.read_csv(input_path, index_col=None)        
        table_parameter_list = []
        if test_environment == "dev":
            copy_dataset_source_params = 'FALSE'
            copy_dataset_target_params = ',"target_copy_dataset":"TRUE"'
        else:
            copy_dataset_target_params = ',"target_copy_dataset":"FALSE"'
        for index, row in df.iterrows():
            try:
                database_name = str(row['Database']).strip().lower()
                table_name = str(row['Table']).strip().lower()
                is_table_to_be_tested = row['ToBeTested']

                if is_table_to_be_tested:
                    backdate_date = testing_date
                else:
                    backdate_date = (datetime.strptime(testing_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
                # query = "Select LOWER(columnname) AS columnname "\
                #     "From dbc.columnsV WHERE LOWER(DatabaseName) = '{}' AND LOWER(TableName) = '{}'"\
                #     ";".format(database_name, table_name)

                # results_df = pd.read_sql(query, td_engine)
                # valid_to_date_present = 'valid_to_date' in results_df['columnname'].unique()
                if is_table_to_be_tested:
                    update_condition = ',"update":"TRUE"'
                else:
                    update_condition = ',"update":"FALSE"'

                if test_environment != 'dev':
                    if database_name == 'edwhr_staging' and table_name == 'ref_sk_xwlk':
                        copy_dataset_source_params = 'FALSE'
                    elif database_name == 'edwhr' and table_name in ['ref_email_to_hr_status' , 'junc_recruitment_job_board' , 'ref_percentage_range' , 'ref_posting_status', 'ref_posting_board_type']:
                        copy_dataset_source_params = 'TRUE'
                    elif database_name == 'edwhr_staging':
                        copy_dataset_source_params = 'TRUE'
                    elif database_name != 'edwhr_staging':
                        copy_dataset_source_params = 'FALSE'
                    else:
                        copy_dataset_source_params = 'FALSE'

                if database_name == 'edwhr_staging' and test_environment == 'dev' and not table_name in ('ref_sk_type','ref_sk_xwlk'):
                    continue
                else:
                    table_parameter_list.append("{"
                    +'"db":"{}","table":"{}","date":"{}"'.format(database_name, table_name,backdate_date) 
                    + update_condition + ',"use_copy_dataset":"{}"'.format(copy_dataset_source_params) + copy_dataset_target_params + "}")
            except Exception as e1:
                print(e1)
                print("Database : {}, Table : {}".format(database_name,table_name))
                pass
        
        if len(table_parameter_list) > 0:
            # print(table_parameter_list)
            full_table_parameter_json_list = []
            full_table_parameter_json_list.append("[")
            full_table_parameter_json_list.append(",\n".join(table_parameter_list))
            full_table_parameter_json_list.append("]")
            write_file_local(output_path, full_table_parameter_json_list)
        dt2 = datetime.now()
        print(dt2-dt1)
        # print("Tables for Record Counts : ", len(df))

In [ ]:
print("Begin of Processing")

gen_copy_tables_parameters()

print("End of Processing")